<a href="https://colab.research.google.com/github/suyash1574/GEN-AI-Workshop/blob/main/src/day2/notebooks/03_mini_rag_lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mini RAG Lab

In this lab, we will implement a production-grade Retrieval-Augmented Generation (RAG) system. The goal is to retrieve relevant information from a set of documents and use it to enhance the responses generated by a language model.

## What You'll Learn:
- Document chunking strategies for better retrieval
- Embedding generation and similarity search
- Multi-document retrieval with ranking
- Context-aware response generation
- Production RAG pipeline best practices

In [1]:
# Import necessary libraries for RAG with chunking
import json
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import warnings
import re
from typing import List

warnings.filterwarnings('ignore')

# Check if GPU is available (industry standard practice)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# 🎯 LEARNING OBJECTIVE: Understanding the RAG Pipeline
# In this lab, you'll build a complete RAG system that includes:
# 1. Document chunking (breaking large texts into smaller pieces)
# 2. Embedding generation (converting text to numerical vectors)
# 3. Similarity-based retrieval (finding relevant documents)
# 4. Context-aware response generation (using retrieved info to answer)

Using device: cuda


## Step 1: Load the Data

We will load the handbook text and the FAQ dataset to use in our RAG system.

In [2]:
# Load handbook text
with open('/content/handbook_text.txt', 'r') as file:
    handbook_text = file.read()

# Load FAQ dataset from JSON format
data_path = '/content/campus_faq.json'

try:
    with open(data_path) as f:
        data = json.load(f)
    print("✓ Dataset loaded successfully!")
except FileNotFoundError:
    print("❌ Error: Could not find the dataset file. Please check the path.")
    raise

# Extract questions and answers from the nested structure
questions = []
answers = []

for item in data['faq']:
    questions.append(item['question'])
    answers.append(item['answer'])

# Create DataFrame for easier data manipulation
faq_data = pd.DataFrame({
    'question': questions,
    'answer': answers
})

print(f"Dataset contains {len(faq_data)} question-answer pairs")
print(f"Handbook text length: {len(handbook_text)} characters")

✓ Dataset loaded successfully!
Dataset contains 30 question-answer pairs
Handbook text length: 1924 characters


## Step 2: Document Chunking Strategy

**Why do we need chunking?**

In production RAG systems, we don't just dump entire documents into our knowledge base. Here's why:

1. **Memory Efficiency**: Large documents create huge embeddings that consume memory
2. **Retrieval Precision**: Smaller chunks mean more focused, relevant results
3. **Context Quality**: Generation models work better with concise, relevant context
4. **Scalability**: Chunking allows us to handle massive document collections

**Industry Best Practices for Chunking:**
- **Chunk Size**: Typically 100-500 tokens (we'll use ~400 characters)
- **Overlap**: 10-20% overlap between chunks to maintain context
- **Boundary Awareness**: Split on sentences, not mid-word
- **Metadata**: Track which document each chunk came from

In [4]:
def chunk_text(text: str, chunk_size: int = 500, overlap: int = 50) -> List[str]:
    """
    Split text into overlapping chunks for better retrieval.
    This is a simplified version - production systems use more sophisticated methods.

    Args:
        text: Input text to chunk
        chunk_size: Maximum characters per chunk
        overlap: Characters to overlap between chunks

    Returns:
        List of text chunks
    """
    # TODO: Split text into sentences (hint: use re.split with pattern r'[.!?]+')
    sentences = re.split(r'[.!?]+', text)

    chunks = []
    current_chunk = ""

    for sentence in sentences:
        sentence = sentence.strip()
        if not sentence:
            continue

        # TODO: Check if adding this sentence would exceed chunk_size
        # If yes AND current_chunk is not empty, start a new chunk
        if len(current_chunk) + len(sentence) > chunk_size and current_chunk:
            chunks.append(current_chunk.strip())

            # Add overlap from the end of previous chunk
            if overlap > 0 and len(current_chunk) > overlap:
                current_chunk = current_chunk[-overlap:] + " " + sentence
            else:
                current_chunk = sentence
        else:
            # TODO: Add sentence to current_chunk (handle empty chunk case)
            if current_chunk:
                current_chunk += " " + sentence
            else:
                current_chunk = sentence

    # Don't forget the last chunk!
    if current_chunk:
        chunks.append(current_chunk.strip())

    return chunks

# Test your chunking function
print("🧪 Testing your chunking function...")
sample_text = "This is sentence one. This is sentence two. This is sentence three. This is sentence four."
test_chunks = chunk_text(sample_text, chunk_size=50, overlap=10)
print(f"Original: {sample_text}")
print(f"Chunks: {test_chunks}")

# 🎯 Expected output: Should create overlapping chunks that respect sentence boundaries
# If your function works correctly, you should see chunks with some overlap

🧪 Testing your chunking function...
Original: This is sentence one. This is sentence two. This is sentence three. This is sentence four.
Chunks: ['This is sentence one This is sentence two', 'ntence two This is sentence three', 'ence three This is sentence four']


## Step 3: Prepare Your Knowledge Base

**What makes a good knowledge base?**

In production RAG systems, your knowledge base is everything. Here's what matters:

1. **Document Diversity**: Mix of structured (FAQ) and unstructured (handbook) data
2. **Metadata Tracking**: Know where each piece of information came from
3. **Quality Control**: Clean, relevant, up-to-date information
4. **Proper Formatting**: Consistent structure for better retrieval

**Your Knowledge Base will contain:**
- Chunked handbook sections (unstructured text)
- FAQ pairs (structured Q&A)
- Metadata for each document (source, type, etc.)

In [14]:
# Step 3a: Chunk the handbook text
print("Chunking handbook text...")
# TODO: Use your chunking function to split handbook_text into chunks
# Use chunk_size=400 and overlap=50
handbook_chunks = chunk_text(handbook_text, chunk_size=400, overlap=50)
print(f"✓ Created {len(handbook_chunks)} handbook chunks")

# Step 3b: Format FAQ documents
print("Preparing FAQ documents...")
faq_documents = []

# TODO: Loop through faq_data and create formatted documents
# Format each as: "Question: [question]\nAnswer: [answer]"
for _, row in faq_data.iterrows():
    faq_doc = f"Question: {row['question']}\nAnswer: {row['answer']}"
    faq_documents.append(faq_doc)

print(f"✓ Created {len(faq_documents)} FAQ documents")

# Step 3c: Combine and create metadata
all_documents = handbook_chunks + faq_documents
document_metadata = []

# Create metadata for handbook chunks (this is complete)
for i, chunk in enumerate(handbook_chunks):
    document_metadata.append({
        'source': 'handbook',
        'chunk_id': i,
        'type': 'text_chunk'
    })

# TODO: Create metadata for FAQ documents
for i, faq_doc in enumerate(faq_documents):
    metadata = {
        'source': 'faq',
        'faq_id': i,
        'type': 'qa_pair',
        'question': faq_data.iloc[i]['question']
    }
    document_metadata.append(metadata)

print(f"\n📚 Knowledge Base Summary:")
print(f"- Total documents: {len(all_documents)}")
print(f"- Handbook chunks: {len(handbook_chunks)}")
print(f"- FAQ documents: {len(faq_documents)}")
print(f"- Metadata entries: {len(document_metadata)}")

# Let's examine a few examples
print(f"\n🔍 Sample Documents:")
if handbook_chunks:
    print(f"Handbook chunk example: {handbook_chunks[0][:100]}...")
if faq_documents:
    print(f"FAQ document example: {faq_documents[0][:100]}...")

Chunking handbook text...
✓ Created 8 handbook chunks
Preparing FAQ documents...
✓ Created 30 FAQ documents

📚 Knowledge Base Summary:
- Total documents: 38
- Handbook chunks: 8
- FAQ documents: 30
- Metadata entries: 38

🔍 Sample Documents:
Handbook chunk example: # Handbook for AI Workshop

This handbook serves as a reference for participants in the AI workshop ...
FAQ document example: Question: What are the library hours?
Answer: The library is open from 8 AM to 10 PM, Monday to Frid...


## Step 4: Create Embeddings - The Heart of RAG

**Understanding Embeddings in RAG:**

Embeddings are the magic that makes RAG work. Here's the concept:

1. **Text → Numbers**: Convert human language into vectors that computers can process
2. **Semantic Similarity**: Similar meanings result in similar vectors
3. **Mathematical Operations**: We can calculate similarity using cosine similarity
4. **Efficiency**: Once created, embeddings enable fast similarity searches

**Why 'all-MiniLM-L6-v2'?**
- **Size**: Only 22MB - perfect for learning and small deployments
- **Quality**: Trained on diverse text, good general-purpose performance
- **Speed**: Fast inference, works well on CPU and GPU
- **Dimensions**: 384-dimensional vectors - good balance of information and efficiency

**Production Considerations:**
- Larger models (768+ dimensions) for better accuracy
- Domain-specific models for specialized content
- Multilingual models for international applications
- Regular re-embedding when content changes

In [18]:
# Initialize the sentence transformer model
print("🔧 Loading sentence transformer model...")
sentence_model = SentenceTransformer('all-MiniLM-L6-v2')
print("✓ Model loaded successfully!")

# TODO: Create embeddings for all documents
print(f"\n🧮 Creating embeddings for {len(all_documents)} documents...")
print("This might take a moment - we're converting text to mathematical vectors!")

# YOUR CODE HERE: Use sentence_model.encode() to create embeddings
# Hint: Use show_progress_bar=True to see progress
document_embeddings = sentence_model.encode(all_documents, show_progress_bar=True)

print(f"✓ Document embeddings created!")
print(f"📊 Embedding Statistics:")
print(f"   - Shape: {document_embeddings.shape}")
print(f"   - Each document → {document_embeddings.shape[1]} numbers")
print(f"   - Memory usage: ~{document_embeddings.nbytes / 1024 / 1024:.1f} MB")

# 🧪 Let's explore what embeddings look like
print(f"\n🔬 Embedding Analysis:")
print(f"First document embedding (first 10 values): {document_embeddings[0][:10]}")
print(f"Embedding range: {document_embeddings.min():.3f} to {document_embeddings.max():.3f}")

# TODO: Calculate similarity between first two documents
# Hint: Use cosine_similarity([document_embeddings[0]], [document_embeddings[1]])[0][0]
similarity = cosine_similarity([document_embeddings[0]], [document_embeddings[1]])[0][0]
print(f"Similarity between first two documents: {similarity:.3f}")

# 🎯 Understanding: What does this similarity score mean?
# - 1.0 = identical documents
# - 0.0 = completely unrelated
# - -1.0 = opposite meaning (rare in practice)

🔧 Loading sentence transformer model...


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✓ Model loaded successfully!

🧮 Creating embeddings for 38 documents...
This might take a moment - we're converting text to mathematical vectors!


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

✓ Document embeddings created!
📊 Embedding Statistics:
   - Shape: (38, 384)
   - Each document → 384 numbers
   - Memory usage: ~0.1 MB

🔬 Embedding Analysis:
First document embedding (first 10 values): [-0.02470068 -0.04378792  0.03602071 -0.00156944  0.01082408  0.05379568
  0.05765799 -0.00783203 -0.05399619  0.02268663]
Embedding range: -0.211 to 0.217
Similarity between first two documents: 0.665


## Step 5: Implement Smart Retrieval

**The Retrieval Challenge:**

When a user asks "What are the library hours?", your system needs to:

1. **Understand the query**: Convert question to same vector space as documents
2. **Search efficiently**: Compare query against all document embeddings
3. **Rank results**: Sort by relevance (similarity scores)
4. **Filter quality**: Only return documents above a confidence threshold
5. **Provide context**: Return enough information but not too much

**Key Parameters:**
- **top_k**: How many documents to retrieve (typically 3-5)
- **similarity_threshold**: Minimum confidence score (0.1-0.3 is common)
- **diversity**: Avoid returning very similar documents

In [19]:
def retrieve_relevant_documents(query: str, sentence_model, document_embeddings, all_documents, document_metadata, top_k: int = 3, similarity_threshold: float = 0.1):
    """
    Retrieve the most relevant documents for a given query.
    This function demonstrates the retrieval process step-by-step.
    """
    print(f"🔍 RETRIEVAL PROCESS for: '{query}'")
    print("=" * 50)

    # Step 1: Convert query to embedding
    print("Step 1: Converting query to embedding...")
    # TODO: Create embedding for the query using sentence_model
    query_embedding = sentence_model.encode(query, convert_to_tensor=True).to(device)
    print(f"✓ Query converted to {query_embedding.shape[0]}-dimensional vector")

    # Step 2: Calculate similarities
    print("Step 2: Calculating similarities with all documents...")
    # TODO: Calculate cosine similarities between query and all documents
    similarities = cosine_similarity([query_embedding.cpu()], document_embeddings)[0]
    print(f"✓ Calculated {len(similarities)} similarity scores")
    print(f"   Similarity range: {similarities.min():.3f} to {similarities.max():.3f}")

    # Step 3: Find top-k most similar documents
    print(f"Step 3: Finding top-{top_k} most relevant documents...")
    # TODO: Get indices of top-k highest similarity scores
    # Hint: Use np.argsort(similarities)[::-1][:top_k]
    top_indices = np.argsort(similarities)[::-1][:top_k]
    print(f"✓ Top document indices: {top_indices}")

    # Step 4: Filter by threshold and prepare results
    print(f"Step 4: Filtering by threshold ({similarity_threshold})...")
    results = []

    for idx in top_indices:
        score = similarities[idx]
        # TODO: Check if score meets threshold
        if score >= similarity_threshold:
            result = {
                'document': all_documents[idx],
                'score': float(score),
                'metadata': document_metadata[idx],
                'index': int(idx)
            }
            results.append(result)

    print(f"✓ {len(results)} documents passed the threshold")

    # Step 5: Display results for learning
    print(f"\n📋 RETRIEVAL RESULTS:")
    for i, result in enumerate(results):
        source = result['metadata']['source']
        score = result['score']
        preview = result['document'][:80] + "..." if len(result['document']) > 80 else result['document']
        print(f"  {i+1}. [{source.upper()}] Score: {score:.3f}")
        print(f"      Preview: {preview}")
        print()

    return results

# 🧪 Test your retrieval function
print("🧪 Testing Your Retrieval Function")
print("=" * 40)
test_queries = [
    "What are the library hours?",
    "How do I register for classes?"
]

for query in test_queries:
    retrieved_docs = retrieve_relevant_documents(query, sentence_model, document_embeddings, all_documents, document_metadata, top_k=3)
    print(f"Retrieved {len(retrieved_docs)} documents for: '{query}'\n")

# 🎯 Success Criteria:
# - Should return 3 documents per query (if above threshold)
# - Scores should be between 0 and 1
# - Higher scores should appear first
# - Should show a mix of FAQ and handbook sources

🧪 Testing Your Retrieval Function
🔍 RETRIEVAL PROCESS for: 'What are the library hours?'
Step 1: Converting query to embedding...
✓ Query converted to 384-dimensional vector
Step 2: Calculating similarities with all documents...
✓ Calculated 38 similarity scores
   Similarity range: -0.042 to 0.798
Step 3: Finding top-3 most relevant documents...
✓ Top document indices: [ 8 35 19]
Step 4: Filtering by threshold (0.1)...
✓ 3 documents passed the threshold

📋 RETRIEVAL RESULTS:
  1. [FAQ] Score: 0.798
      Preview: Question: What are the library hours?
Answer: The library is open from 8 AM to 1...

  2. [FAQ] Score: 0.554
      Preview: Question: Where can I study late at night?
Answer: The library has 24-hour study...

  3. [FAQ] Score: 0.482
      Preview: Question: What are the dining hours?
Answer: The cafeteria is open from 7 AM to ...

Retrieved 3 documents for: 'What are the library hours?'

🔍 RETRIEVAL PROCESS for: 'How do I register for classes?'
Step 1: Converting query to emb

## Step 6: Response Generation - Bringing It All Together

**The Generation Challenge:**

Now you have relevant documents, but how do you create a helpful response?

**Key Considerations:**
1. **Prompt Engineering**: How you structure the input affects output quality
2. **Context Management**: Too little context = poor answers, too much = confusion
3. **Generation Parameters**: Temperature, max_length affect creativity vs focus
4. **Response Extraction**: Separate your generated answer from the prompt

**Production Tips:**
- Always include retrieved context in your prompt
- Use clear instructions ("Based on the context...")
- Set reasonable token limits
- Handle cases where no good context is found

In [20]:
# Load GPT-2 model and tokenizer for response generation
print("🤖 Loading GPT-2 model for text generation...")
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-medium')

# Add padding token if it doesn't exist
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

generation_model = GPT2LMHeadModel.from_pretrained('gpt2-medium').to(device)
print("✓ Generation model loaded successfully!")
print(f"Model running on: {next(generation_model.parameters()).device}")

🤖 Loading GPT-2 model for text generation...


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

✓ Generation model loaded successfully!
Model running on: cuda:0


In [21]:
def generate_rag_response(query: str, sentence_model, document_embeddings, all_documents, document_metadata, generation_model, tokenizer, max_new_tokens: int = 100):
    """
    Complete RAG pipeline: retrieve relevant documents and generate response.
    🛠️ YOUR TASK: Complete the missing parts of this function!
    """
    print(f"\n🎯 RAG PIPELINE for: '{query}'")
    print("=" * 60)

    # Step 1: Retrieve relevant documents
    print("STEP 1: RETRIEVAL")
    retrieved_docs = retrieve_relevant_documents(query, sentence_model, document_embeddings, all_documents, document_metadata, top_k=3)

    # Step 2: Prepare context
    print("STEP 2: CONTEXT PREPARATION")
    if not retrieved_docs:
        print("⚠️ No relevant documents found - generating without context")
        context = "No specific information found in the knowledge base."
    else:
        # TODO: Combine retrieved documents into context
        context_parts = []
        for doc in retrieved_docs:
            source_label = f"[{doc['metadata']['source'].upper()}]"
            # YOUR CODE HERE: Add formatted document to context_parts
            # Format: f"{source_label} {doc['document']}"
            context_parts.append(f"{source_label} {doc['document']}")


        context = "\n\n".join(context_parts)

    print(f"✓ Context prepared ({len(context)} characters)")

    # Step 3: Create prompt
    print("STEP 3: PROMPT ENGINEERING")
    # TODO: Create a well-structured prompt
    # Include: context, user question, and clear instructions
    prompt = f"""Based on the following context, please answer the user's question accurately and helpfully.

Context:
{context}

User Question: {query}

Answer:"""

    print(f"✓ Prompt created ({len(prompt)} characters)")

    # Step 4: Generate response
    print("STEP 4: RESPONSE GENERATION")
    # TODO: Tokenize the prompt and move to device
    inputs = tokenizer.encode(prompt, return_tensors='pt').to(device)

    with torch.no_grad():
        # TODO: Generate response using the model
        outputs = generation_model.generate(
            inputs,
            max_new_tokens=max_new_tokens,
            temperature=0.7,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id
        )

    # Step 5: Extract and clean response
    print("STEP 5: RESPONSE EXTRACTION")
    full_response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # TODO: Extract just the generated part (after the prompt)
    generated_answer = full_response[len(prompt):].strip()


    print(f"✓ Response generated ({len(generated_answer)} characters)")

    return {
        'query': query,
        'retrieved_documents': retrieved_docs,
        'context_used': context,
        'generated_answer': generated_answer,
        'num_docs_retrieved': len(retrieved_docs)
    }

# 🧪 Test your complete RAG system!
print("🚀 Testing Your Complete RAG System!")
print("If you've implemented everything correctly, this should work:")

# Uncomment the line below once you've completed the function
result = generate_rag_response("What are the library hours?", sentence_model, document_embeddings, all_documents, document_metadata, generation_model, tokenizer)
print(f"\n🎉 FINAL ANSWER: {result['generated_answer']}")

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


🚀 Testing Your Complete RAG System!
If you've implemented everything correctly, this should work:

🎯 RAG PIPELINE for: 'What are the library hours?'
STEP 1: RETRIEVAL
🔍 RETRIEVAL PROCESS for: 'What are the library hours?'
Step 1: Converting query to embedding...
✓ Query converted to 384-dimensional vector
Step 2: Calculating similarities with all documents...
✓ Calculated 38 similarity scores
   Similarity range: -0.042 to 0.798
Step 3: Finding top-3 most relevant documents...
✓ Top document indices: [ 8 35 19]
Step 4: Filtering by threshold (0.1)...
✓ 3 documents passed the threshold

📋 RETRIEVAL RESULTS:
  1. [FAQ] Score: 0.798
      Preview: Question: What are the library hours?
Answer: The library is open from 8 AM to 1...

  2. [FAQ] Score: 0.554
      Preview: Question: Where can I study late at night?
Answer: The library has 24-hour study...

  3. [FAQ] Score: 0.482
      Preview: Question: What are the dining hours?
Answer: The cafeteria is open from 7 AM to ...

STEP 2: CONTEX

## Step 7: Comprehensive Testing

Let's test our RAG system with various types of queries to see how it performs.

In [22]:
# 🎓 FINAL CHALLENGE: Test Your Complete RAG System

test_queries = [
    "What are the library hours?",
    "How do I register for classes?",
    "What dining options are available on campus?",
    "Tell me about student support services",
    "What happens if I lose my student ID?",
    "How can I get involved in campus activities?"
]

print("🏆 COMPREHENSIVE RAG SYSTEM TESTING")
print("=" * 60)
print("🛠️ YOUR TASK: Uncomment and run the testing code once everything is implemented!")

# TODO: Uncomment the code below once you've completed all the functions


all_results = []

for i, query in enumerate(test_queries, 1):
    print(f"\n📝 TEST {i}: {query}")
    print("-" * 50)

    result = generate_rag_response(query, sentence_model, document_embeddings, all_documents, document_metadata, generation_model, tokenizer)
    all_results.append(result)

    print(f"\n📊 RESULTS SUMMARY:")
    print(f"- Documents retrieved: {result['num_docs_retrieved']}")
    print(f"- Context length: {len(result['context_used'])} characters")
    print(f"- Generated answer: {result['generated_answer']}")

    if i < len(test_queries):
        print("\n" + "="*60)

print(f"\n✅ Testing complete! ")


print(f"\n🎯 ANALYSIS QUESTIONS (Discuss with your team):")
print("- Which queries worked best? Why?")
print("- Where did the system struggle? What could be the reasons?")
print("- How could you improve the chunking strategy?")
print("- What about the prompts could be enhanced?")
print("- How might you handle queries with no relevant context?")
print("- What would you change for a production system?")

print(f"\n🏅 BONUS CHALLENGES:")
print("1. Implement a similarity threshold that adapts based on query complexity")
print("2. Add a re-ranking step that considers document diversity")
print("3. Implement caching for embeddings to speed up repeated queries")
print("4. Add evaluation metrics to measure RAG system quality")

🏆 COMPREHENSIVE RAG SYSTEM TESTING
🛠️ YOUR TASK: Uncomment and run the testing code once everything is implemented!

📝 TEST 1: What are the library hours?
--------------------------------------------------

🎯 RAG PIPELINE for: 'What are the library hours?'
STEP 1: RETRIEVAL
🔍 RETRIEVAL PROCESS for: 'What are the library hours?'
Step 1: Converting query to embedding...
✓ Query converted to 384-dimensional vector
Step 2: Calculating similarities with all documents...
✓ Calculated 38 similarity scores
   Similarity range: -0.042 to 0.798
Step 3: Finding top-3 most relevant documents...
✓ Top document indices: [ 8 35 19]
Step 4: Filtering by threshold (0.1)...
✓ 3 documents passed the threshold

📋 RETRIEVAL RESULTS:
  1. [FAQ] Score: 0.798
      Preview: Question: What are the library hours?
Answer: The library is open from 8 AM to 1...

  2. [FAQ] Score: 0.554
      Preview: Question: Where can I study late at night?
Answer: The library has 24-hour study...

  3. [FAQ] Score: 0.482
      

In [17]:
{"interactive_code_output":{"__jupyter_types__":"interactive_code_output","outputs":[]}}

{'interactive_code_output': {'__jupyter_types__': 'interactive_code_output',
  'outputs': []}}